In [38]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd    
import io
import wikipedia as w
import re

# Football Stadiums

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_football_stadiums_in_England"
#url of page containing list of football stadiums
r = requests.get(url)
soup = BS(r.text)    #BS objecct for the url


/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [3]:
stadium_list = soup.find_all('table',{'class' : 'wikitable'})
stadium_list = stadium_list[0] #2 tables only first table
stadium_list = stadium_list.find('tbody') #body of table selected
stadium_list = stadium_list.find_all('tr') #select all rows

stadiums = []

for i in stadium_list:
    tmp = i.find('a')
    if tmp != None:
        stadiums.append(tmp.text)

print stadiums[:10]   #print first 10 stadiums

[u'Wembley Stadium', u'Tottenham Hotspur', u'Old Trafford', u'Emirates Stadium', u'London Stadium', u'City of Manchester Stadium', u'Anfield', u"St James' Park", u'Stadium of Light', u'Villa Park']


In [4]:
cnt = 0
ix = 0
# The following loop uses wikipedia api (which is shit), to get 
# wikipedia pg object (we pass title as argument) .Sometimes one title 
# can have multiple disambiguation , this will be caught as exception
# and we can print all possible alternate titles. We can then pass the
# appropriate string .

# pg.content prints entire string , print it and you will see 
# '== %title% ==' for level 2 headers and '=== %title% ===' for level 
# 3 headers . I have divided it such that text docs are seperated by 
# level 2 headers , and any text doc having less than 200 words is discarded

cnt = 0
ix = 0
while ix < len(stadiums):
    try :
        pg = w.WikipediaPage(stadiums[ix])  # pg object contains info about wiki page of stad[ix]
        url =  pg.url                       # gives url of wiki page

        r = requests.get(url)               
        soup = BS(r.text)                   # soup object of wikipage
        
        title = soup.find('h1').text        #h1 is the title of page(this is first heading , getting it through BS4 as not included in pg.content)
        print title                         #prints stadium name 

        string = pg.content                       # whole page as unicode string
        pattern = r'\n==\s.+\s==\n'               # Regex pattern for headings 
        headings  = re.findall(pattern, string)   # Each unnique heading 
        
        
        # ind is a list which contains tuples as entries 
        # tuple has info (heading name,index of start )
        # heading name modified with _ and exchanged '/' for '\'
        # as these can give problems 
        ind = [('_'+title.strip('\n').strip('=').replace('/','\\').replace(' ','_')+'_title',0)]
        # ^^ we explicitly give first heading and index 0 ,as not part of pg.content
        
        # we use below loop to get doc_text which contains all words
        # inside headings
        for i in headings :
            pos = string.find(i)
            i = i.strip('\n').strip('=').replace('/','\\').replace(' ','_')
            ind.append((i,pos))
        ind.append((None,-1))
        
        page_docs = 0   # num of docs > 180 in length per title
                        # just so we get idea how many page_docs
                        # qualified at each iteration
        for i in range(len(ind)-1) :
            
            doc_text =  string[ind[i][1]:ind[i+1][1]-1]
            # doc_text is content under heading
            # if doc_text has more than 180 words ,
            # make a document and store in folder
            # store file as 
            # your_Directory/Stadium_%stadium_name%_%topic_name%
            
#             print 'name of document '+ ind[i][0]
#             print len(doc_text.split())
            if len(doc_text.split()) >= 180 :
                f = io.open("/home/kush/Desktop/Wiki_Data_2/" + 'Stadium_' + title.replace(' ','_') + ind[i][0], "w+", encoding = 'utf-8')
                f.write(doc_text)
                f.close()
                cnt += 1
                page_docs +=1
        print page_docs
        print cnt
        ix = ix+1
                
    except Exception as e:         # IMP
        print '!'*100              # also try by uncommenting below
        print 'EXCEPTION!!!'       # section     
        ix += 1
#        print e     
        continue
#         print 'Enter another string'  # Enter one of the strings from the disambiguation   
#         inp = raw_input()
#         if inp == 'n':
#             ix = ix + 1
#             continue
#         else : 
#             stadiums[ix] = inp
#             continue
print 'number of docs = '+ str(cnt)

Wembley Stadium
6
6
Tottenham Hotspur F.C.


KeyboardInterrupt: 

# Dog Breeds

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_dog_breeds"
r = requests.get(url)
soup = BS(r.text)

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [5]:
dog_list = soup.find_all('table',{'class' : 'wikitable'})
dog_list = dog_list[0].find_all('tr') # Fetch all rows
nrows = len(dog_list)

In [6]:
dogs = []
for i in dog_list[1: nrows -1]:
    tmp = (i.find('td').find('a')).text
    dogs.append(tmp)

print dogs[:10]

[u'Affenpinscher', u'Afghan Hound', u'Afghan Shepherd', u'Aidi', u'Airedale Terrier', u'Akbash', u'Akita', u'Alano Espa\xf1ol', u'Alaskan husky', u'Alaskan Klee Kai']


In [8]:
cnt = 917
ix = 444
while ix < len(dogs):
    try :
        pg = w.WikipediaPage(dogs[ix])  # pg object contains info about wiki page of dog[ix]
        url =  pg.url                       # gives url of wiki page

        r = requests.get(url)               
        soup = BS(r.text)                   # soup object of wikipage
        
        title = soup.find('h1').text        #h1 is the title of page(this is first heading , getting it through BS4 as not included in pg.content)
        print title                         #prints dog name 

        string = pg.content                       # whole page as unicode string
        pattern = r'\n==\s.+\s==\n'               # Regex pattern for headings 
        headings  = re.findall(pattern, string)   # Each unnique heading 
        
        
        # ind is a list which contains tuples as entries 
        # tuple has info (heading name,index of start )
        # heading name modified with _ and exchanged '/' for '\'
        # as these can give problems 
        ind = [('_'+title.strip('\n').strip('=').replace('/','\\').replace(' ','_')+'_title',0)]
        # ^^ we explicitly give first heading and index 0 ,as not part of pg.content
        
        # we use below loop to get doc_text which contains all words
        # inside headings
        for i in headings :
            pos = string.find(i)
            i = i.strip('\n').strip('=').replace('/','\\').replace(' ','_')
            ind.append((i,pos))
        ind.append((None,-1))
        
        page_docs = 0   # num of docs > 180 in length per title
                        # just so we get idea how many page_docs
                        # qualified at each iteration
        for i in range(len(ind)-1) :
            
            doc_text =  string[ind[i][1]:ind[i+1][1]-1]
            # doc_text is content under heading
            # if doc_text has more than 180 words ,
            # make a document and store in folder
            # store file as 
            # your_Directory/Dog_%breed_name%_%topic_name%
            
#             print 'name of document '+ ind[i][0]
#             print len(doc_text.split())
            if len(doc_text.split()) >= 180 :
                f = io.open("/home/kush/Desktop/Wiki_Data_2/Dogs/" + 'Dog_' + title.replace(' ','_') + ind[i][0], "w+", encoding = 'utf-8')
                f.write(doc_text)
                f.close()
                cnt += 1
                page_docs +=1
        print page_docs
        print cnt
        ix = ix+1
                
    except Exception as e:         # IMP
        print '!'*100              # also try by uncommenting below
        print 'EXCEPTION!!!'       # section     
        ix += 1
        print e     
#         continue
        print 'Enter another string'  # Enter one of the strings from the disambiguation   
        inp = raw_input()             # or 'n' to skip
        if inp == 'n':
            ix = ix + 1
            continue
        else : 
            dogs[ix] = inp
            continue
print 'number of docs = '+ str(cnt) 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
EXCEPTION!!!
Page id "Slovakian Wirehaired Pointer" does not match any pages. Try another id!
Enter another string
Slovak Rough-haired Pointer
Slovak Rough-haired Pointer
1
918
Smaland Hound
2
920
Small Greek Domestic Dog
1
921
Soft-coated Wheaten Terrier
4
925
South Russian Ovcharka
0
925
Southern Hound
1
926
Spanish Mastiff
1
927
Spanish Water Dog
2
929
Spinone Italiano
3
932
Sporting Lucas Terrier
0
932
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
EXCEPTION!!!
"Saint Bernard" may refer to: 
St. Bernard (dog)
Bernard of Vienne
Bernard of Menthon
Bernard of Thiron
Bernard degli Uberti
Bernard of Clairvaux
Bernardo Tolomei
Bernard of Corleone
Saint-Bernard-de-Lacolle, Quebec
Saint-Bernard, Quebec
Saint-Bernard-de-Michaudville, Quebec
L'Isle-aux-Coudres, Quebec
St. Bernard, Nova Scotia
Saint-Bernard, Ain
Saint-Bernard, Côte-d'Or
Sai

In [7]:
dogs.index('Slovakian Wirehaired Pointer')

444

# Geographies of Countries

In [9]:
url = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"
r = requests.get(url)
soup = BS(r.text)

In [13]:
country_list = soup.find_all('table',{'class' : 'wikitable'})
country_list = country_list[0].find_all('tr') # Fetch all rows
print len(country_list)

241


In [19]:
countries= []
for row in country_list :
    name = row.find('a')
    if name != None :
        countries.append(name.text)
        
print len(countries)

240


In [20]:
cnt = 0
ix = 0
while ix < len(countries):
    try :
        pg = w.WikipediaPage(countries[ix])  # pg object contains info about wiki page of dog[ix]
        url =  pg.url                       # gives url of wiki page

        r = requests.get(url)               
        soup = BS(r.text)                   # soup object of wikipage
        
        title = soup.find('h1').text        #h1 is the title of page(this is first heading , getting it through BS4 as not included in pg.content)
        print title                         #prints dog name 

        string = pg.content                       # whole page as unicode string
        pattern = r'\n==\s.+\s==\n'               # Regex pattern for headings 
        headings  = re.findall(pattern, string)   # Each unnique heading 
        
        
        # ind is a list which contains tuples as entries 
        # tuple has info (heading name,index of start )
        # heading name modified with _ and exchanged '/' for '\'
        # as these can give problems 
        ind = [('_'+title.strip('\n').strip('=').replace('/','\\').replace(' ','_')+'_title',0)]
        # ^^ we explicitly give first heading and index 0 ,as not part of pg.content
        
        # we use below loop to get doc_text which contains all words
        # inside headings
        for i in headings :
            pos = string.find(i)
            i = i.strip('\n').strip('=').replace('/','\\').replace(' ','_')
            ind.append((i,pos))
        ind.append((None,-1))
        
        page_docs = 0   # num of docs > 180 in length per title
                        # just so we get idea how many page_docs
                        # qualified at each iteration
        for i in range(len(ind)-1) :
            
            doc_text =  string[ind[i][1]:ind[i+1][1]-1]
            # doc_text is content under heading
            # if doc_text has more than 180 words ,
            # make a document and store in folder
            # store file as 
            # your_Directory/Dog_%breed_name%_%topic_name%
            
#             print 'name of document '+ ind[i][0]
#             print len(doc_text.split())
            if len(doc_text.split()) >= 180 :
                f = io.open("/home/kush/Desktop/Wiki_Data_2/Geography/" + 'Geography_' + title.replace(' ','_') + ind[i][0], "w+", encoding = 'utf-8')
                f.write(doc_text)
                f.close()
                cnt += 1
                page_docs +=1
        print page_docs
        print cnt
        ix = ix+1
                
    except Exception as e:         # IMP
        print '!'*100              # also try by uncommenting below
        print 'EXCEPTION!!!'       # section     
        ix += 1
#         print e     
        continue
#         print 'Enter another string'  # Enter one of the strings from the disambiguation   
#         inp = raw_input()             # or 'n' to skip
#         if inp == 'n':
#             ix = ix + 1
#             continue
#         else : 
#             countries[ix] = inp
#             continue
print 'number of docs = '+ str(cnt) 

China
11
11
India
10
21
United States
13
34
Indonesia
8
42
Brazil
9
51
Pakistan
10
61
Nigeria
9
70
Bangladesh
8
78
Russia
8
86
Japan
12
98
Mexico
9
107
Ethiopia
13
120
Philippines
11
131
Egypt
1
132
Vietnam
11
143
Democratic Republic of the Congo
8
151
Germany
7
158
Iran
9
167
Turkey
8
175
Thailand
14
189
France
7
196
United Kingdom
9
205
Italy
8
213
South Africa
9
222
Tanzania
8
230
Myanmar
9
239
South Korea
11
250
Kenya
8
258
Colombia
9
267
Spain
8
275
Argentina
8
283
Algeria
8
291
Ukraine
7
298
Sudan
9
307
Iraq
8
315
Uganda
9
324
Poland
7
331
Canada
8
339
Morocco
10
349
Saudi Arabia
10
359
Uzbekistan
13
372
Malaysia
10
382
Venezuela
10
392
Afghanistan
10
402
Peru
9
411
Ghana
12
423
Angola
9
432
Nepal
10
442
Yemen
7
449
Mozambique
8
457
Madagascar
10
467
North Korea
9
476
Australia
8
484
Ivory Coast
8
492
Cameroon
8
500
Taiwan
13
513
Niger
7
520
Sri Lanka
10
530
Burkina Faso
9
539
Romania
8
547
Mali
8
555
Syria
8
563
Kazakhstan
9
572
Malawi
8
580
Chile
10
590
Guatemala
8
598
Netherla

#  List of World Heritage Sites

In [21]:
#INDIA
url = 'https://en.wikipedia.org/wiki/List_of_World_Heritage_Sites_in_India'
r = requests.get(url)
soup = BS(r.text)

In [22]:
ind_list = soup.find_all('table',{'class' : 'wikitable'})
ind_list = ind_list[0].find_all('tr') # Fetch all rows
nrows = len(ind_list)

In [28]:
WHS= []
for row in ind_list :
    name = row.find('a')
    if name != None :
        WHS.append(name.text)
        
print len(WHS)

52


In [31]:
#CANADA
url = 'https://en.wikipedia.org/wiki/List_of_World_Heritage_Sites_in_Canada'
r = requests.get(url)
soup = BS(r.text)

In [32]:
can_list = soup.find_all('table',{'class' : 'wikitable'})
can_list = can_list[0].find_all('tr') # Fetch all rows
nrows = len(can_list)

In [33]:
for row in can_list :
    name = row.find('a')
    if name != None :
        WHS.append(name.text)
        
print len(WHS)

72


In [34]:
print WHS[-5:]

[u'Red Bay Basque Whaling Station', u'Rideau Canal', u'SGang Gwaay', u'Waterton Glacier International Peace Park', u'Wood Buffalo National Park']


In [35]:
#RUSSIA
url = 'https://en.wikipedia.org/wiki/List_of_World_Heritage_Sites_in_Russia'
r = requests.get(url)
soup = BS(r.text)

In [36]:
rus_list = soup.find_all('table',{'class' : 'wikitable'})
rus_list = can_list[0].find_all('tr') # Fetch all rows
nrows = len(rus_list)

In [37]:
for row in can_list :
    name = row.find('a')
    if name != None :
        WHS.append(name.text)
        
print len(WHS)

92
